Requirements: Create an account on [FRED's Website](https://fred.stlouisfed.org/docs/api/api_key.html) and create an API key. Store the key in a local .env file like this FRED_API_KEY=your_actual_key_here_12345

In [1]:
import os
from dotenv import load_dotenv
from fredapi import Fred
import yfinance as yf
import pandas as pd

In [2]:
# This looks for the .env file and loads variable
load_dotenv()
api_key = os.getenv('FRED_API_KEY')
fred = Fred(api_key=api_key)

In [3]:
# Fetch FRED data 
# GDPC1 = GDP, PCEPILFE = Core Inflation, UNRATE = Unemployment, M2SL = M2 Money Stock
gdp = fred.get_series('GDPC1')
inflation = fred.get_series('PCEPILFE')
employment = fred.get_series('UNRATE')
money_supply = fred.get_series('M2SL')

# Combine data
macro_data = pd.DataFrame({
    'GDP': gdp,
    'Core Inflation': inflation,
    'Unemployment Rate': employment,
    'M2 Money Stock': money_supply
})

# Aligning the data to the same frequency (daily) and forward filling missing values
macro_data = macro_data.resample('D').ffill().dropna()

In [4]:
# Fetch S&P 500 data
tickers = ['^GSPC', '^VIX', 'SPY']
sp500 = yf.download(tickers, start='1993-01-29')
gscp_price = sp500['Close']['^GSPC']
vix_price = sp500['Close']['^VIX']
spy_volume = sp500['Volume']['SPY']

market_data = pd.DataFrame({
    'GSPC': gscp_price,
    'VIX': vix_price,
    'SPY Volume': spy_volume
}).dropna()

[*********************100%***********************]  3 of 3 completed


In [7]:
# Join data and save to CSV
master_data = market_data.join(macro_data, how='left')
master_data = master_data.ffill().dropna()
master_data.to_csv('data/master_data.csv')